In [68]:
from collections import deque

import numpy as np
import matplotlib.pyplot as plt

In [76]:
CSV_DIR = "C001H001S0002/"
data_list = deque([None, None, None, None])

In [70]:
r = []

In [71]:
s1, s2 = 20, 14  # 1st, 2nd search window
eta = 13 / np.pi / 180  # criterion on angle

In [72]:
class Vector2D(object):
    def __init__(self):
        self.x = 0.0
        self.y = 0.0


class Gap(object):
    def __init__(self):
        self.gap = 0.0
        self.angle = 0.0


class TP(object):
    def __init__(self):
        self.p1 = 0
        self.p2 = 0
        self.err = 0.0
        self.flag = False

In [73]:
def calc_gap(dx1, dx2):
    gx = Vector2D
    dst = Gap

    gx.x = dx2.x - dx1.x
    gx.y = dx2.y - dx1.y
    dst.gap = np.sqrt(gx.x ** 2 + gx.y ** 2)

    in_prod = dx1.x * dx2.x + dx1.y * dx2.y
    in_prod /= np.sqrt(dx1.x ** 2 + dx1.y ** 2) * np.sqrt(dx2.x ** 2 + dx2.y ** 2)
    if in_prod > 1.0: in_prod /= np.abs(in_prod)

    dst.angle = np.arccos(in_prod)

    out_prod = dx1.x * dx2.y - dx1.y * dx2.x
    if out_prod < 0: dst.angle *= -1

    return dst, in_prod, out_prod

def calc_rot(dx, a):
    dx_rot = Vector2D

    dx_rot.x=dx.x*np.cos(a)-dx.y*np.sin(a)
    dx_rot.y=dx.x*np.sin(a)+dx.y*np.cos(a)

    return  dx_rot

In [77]:
def ptv4t(img):
    pp0, pp1, pp2, pp3 = img[0], img[1], img[2], img[3]

    if pp0 is None or pp1 is None or pp2 is None or pp3 is None:
        return None

    result = np.zeros([pp0.shape[0], 4])

    dx = [Vector2D for _ in range(3)]
    dst = [Gap for _ in range(2)]
    pp12 = [TP for _ in range(pp0.shape[0])]
    npa = -1  # the number of available particle
    ntsp = 0  # the number for tracking the same particle

    # 4 times tracking
    for ii in range(pp0.shape[0]):  # 1st
        flag = True  # init flag for the particle tracking
        cf, cfc = 1e10, 1e10  # arbitrary large number

        for jj in range(pp1.shape[0]):  # 2nd
            dx[0].x = pp1[jj][0] - pp0[ii][0]
            dx[0].y = pp1[jj][1] - pp0[ii][1]

            if np.sqrt(dx[0].x ** 2 + dx[0].y ** 2) <= s1:
                for kk in range(pp2.shape[0]):  # 3rd
                    dx[1].x = pp2[kk][0] - pp1[jj][0]
                    dx[1].y = pp2[kk][1] - pp1[jj][1]

                    dst[0], ip0, op0 = calc_gap(dx[0], dx[1])
                    dx[1] = calc_rot(dx[1], dst[0].angle)  # rot of diff vector

                    if dst[0].gap <= s2 and np.abs(dst[0].angle) <= eta:
                        for ll in range(pp3.shape[0]):  # 4th
                            dx[2].x = pp3[ll][0] - pp2[kk][0]
                            dx[2].y = pp3[ll][1] - pp2[kk][1]

                            dst[1], ip1, op1 = calc_gap(dx[1], dx[2])
                            cfc = dst[0].gap ** 2 + dst[1].gap ** 2

                            if dst[0].gap <= s2 and np.abs(dst[0].angle) <= eta and cfc < cf:
                                cf = cfc
                                if flag: npa += 1

                                pp12[npa].p1 = ii
                                pp12[npa].p2 = jj
                                pp12[npa].err = cf
                                pp12[npa].flag = True
                                flag = False
    # post-processing
    # If different particles track the same particle,
    # giving an error flag to particle which has larger error.
    for ii in range(npa):
        if pp12[ii].flag:
            for jj in range(ii+1, npa):
                if pp12[ii].p2 == pp12[jj].p2:
                    if pp12[ii].err > pp12[jj].err:
                        pp12[ii].flag = False
                        ntsp += 1
                        break
                    else:
                        pp12[jj].flag = False
                        ntsp += 1

    print(f"The number for tracking the same particle: {ntsp}")
    print(f"The number of remaining particle by post processing: {npa - ntsp}\n")

    for ii in range(npa):
        if pp12[ii].flag:
            result[ii][0] = pp0[pp12[ii].p1][0]
            result[ii][1] = pp0[pp12[ii].p1][1]
            result[ii][2] = pp1[pp12[ii].p2][0] - pp0[pp12[ii].p1][0]
            result[ii][3] = pp1[pp12[ii].p2][1] - pp0[pp12[ii].p1][1]

    return result[:npa]


for i in range(3000, 3100, 10):
    data_list.popleft()
    tmp = np.loadtxt(CSV_DIR + f"{i:06}.csv", delimiter=',', skiprows=1)
    data_list.append(tmp)

    r.append(ptv4t(img=data_list))

/Users/loblium/opt/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


The number for tracking the same particle: 0
The number of remaining particle by post processing: -1

The number for tracking the same particle: 0
The number of remaining particle by post processing: -1



KeyboardInterrupt: 

In [75]:
r[5]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])